# The full model building process

In [1]:
%matplotlib inline

## Working with data

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

- PyTorch는 [TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html), [TorchAudio](https://pytorch.org/audio/stable/index.html)과 같은 도메인 특화된 라이브러리를 제공하며 본 예시에는 TorchVision dataset을 사용함
- `torchVision.dataset` 모듈은 CIFAR, COCO 등 현실 비전데이터를 포함하며([full list here](https://pytorch.org/docs/stable/torchvision/datasets.html)) 예제에서는 **FashionMNIST**를 사용함

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

- `dataset`은 `dataLoader`의 매개 변수로써 데이터를 통화시키며, 이러한 처리를 통해 batching, sampling, shuffling 그리고 multiprocess을 제공함
- batch size는 64로 설정함

In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(training_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## Creating models

- 신경망 구조 정의를 위해 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)을 상속한 클래스를 작성함
- 신경망의 각 층은 `__init__` 안에 정의하고, 어떻게 데이터를 통화할 지 `forward` 함수에 정의함 
- 하드웨어 가속을 사용하기 위해 GPU로 인스턴스를 옮김

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### Optimizing the Model Parameters
모델 학습을 위해 [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions>) 과 [optimizer](https://pytorch.org/docs/stable/optim.html)을 구현

In [6]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

단일 학습 반복에서 모델은 학습 데이터의 예측값을 만들고 학습 에러를 줄이기 위해 모델 파라미터를 조절함

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print loss
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

test 데이터셋으로 모델 성능을 확인

In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval() # 평가모드로 모델이 사용됨을 선언
    test_loss, correct = 0, 0
    with torch.no_grad(): # 오차 역전차를 사용하지 않음(속도 샹항)
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습 과정은 여러번 반복된다. 각 epoch에서 모델은 좀 더 나은 예측을 위해 파라미터를 학습한다. 매번 epoch에서 정확도는 오르고 loss는 내려가길 기대한다. 

In [9]:
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300389 [    0/60000]
loss: 2.294711 [ 6400/60000]
loss: 2.288752 [12800/60000]
loss: 2.281989 [19200/60000]
loss: 2.270615 [25600/60000]
loss: 2.257631 [32000/60000]
loss: 2.266520 [38400/60000]
loss: 2.251966 [44800/60000]
loss: 2.235633 [51200/60000]
loss: 2.222870 [57600/60000]
Test Error: 
 Accuracy: 50.7%, Avg loss: 0.034737 

Epoch 2
-------------------------------
loss: 2.227797 [    0/60000]
loss: 2.206081 [ 6400/60000]
loss: 2.191580 [12800/60000]
loss: 2.202026 [19200/60000]
loss: 2.154261 [25600/60000]
loss: 2.143416 [32000/60000]
loss: 2.173409 [38400/60000]
loss: 2.142192 [44800/60000]
loss: 2.125310 [51200/60000]
loss: 2.091708 [57600/60000]
Test Error: 
 Accuracy: 51.8%, Avg loss: 0.032738 

Epoch 3
-------------------------------
loss: 2.102849 [    0/60000]
loss: 2.053435 [ 6400/60000]
loss: 2.024248 [12800/60000]
loss: 2.070426 [19200/60000]
loss: 1.955449 [25600/60000]
loss: 1.962847 [32000/60000]
loss: 2.012152 [38400/

## Saving Models
dictionary 형태로 직렬화하는 것이 모델을 저장하는 일반적인 방법임(모델 파라미터를 포함)

In [10]:
torch.save(model.state_dict(), "data/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models

모델을 읽은 것은 신경망 모델의 구조 재생성을 포함함

In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

읽은 모델은 예측 생성에 활용할 수 있음

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
